In [1]:
from owlready2 import onto_path, get_ontology
import owlready2
import os
import itertools
import xml.etree.ElementTree as ET
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup
import ngram
import editdistance
from py_stringmatching.similarity_measure.monge_elkan import MongeElkan
from py_stringmatching.similarity_measure.needleman_wunsch import NeedlemanWunsch
from py_stringmatching.similarity_measure.smith_waterman import SmithWaterman
from py_stringmatching.similarity_measure.affine import Affine
from py_stringmatching.similarity_measure.bag_distance import BagDistance
from py_stringmatching.similarity_measure.cosine import Cosine
from py_stringmatching.similarity_measure.partial_ratio import PartialRatio
from py_stringmatching.similarity_measure.soft_tfidf import SoftTfIdf
from py_stringmatching.similarity_measure.editex import Editex
from py_stringmatching.similarity_measure.generalized_jaccard import GeneralizedJaccard
from py_stringmatching.similarity_measure.jaro_winkler import JaroWinkler
from py_stringmatching.similarity_measure.levenshtein import Levenshtein
from py_stringmatching.similarity_measure.dice import Dice
from py_stringmatching.similarity_measure.jaccard import Jaccard
from py_stringmatching.similarity_measure.jaro import Jaro
from py_stringmatching.similarity_measure.overlap_coefficient import OverlapCoefficient
from py_stringmatching.similarity_measure.partial_token_sort import PartialTokenSort
from py_stringmatching.similarity_measure.ratio import Ratio
from py_stringmatching.similarity_measure.soundex import Soundex
from py_stringmatching.similarity_measure.tfidf import TfIdf
from py_stringmatching.similarity_measure.token_sort import TokenSort
from py_stringmatching.similarity_measure.tversky_index import TverskyIndex
from re import finditer
from nltk.corpus import wordnet
from itertools import product
from gensim.models import KeyedVectors

In [3]:
path = "C:\\Users\\user\\Desktop\\Data\\ontologies"
os.chdir(path)
ListOfOnto = os.listdir()

In [2]:
model = KeyedVectors.load_word2vec_format("C:/Users/user/Desktop/Data/GoogleNews-vectors-negative300.bin", binary=True)

In [4]:
AllOnto = []
for file in ListOfOnto:
    if file.endswith(".owl"):
        file_path = f"{path}\{file}"
        AllOnto.append(owlready2.get_ontology(file_path).load())
o1 = AllOnto[-8]

In [5]:
def GetDataFrame(onto):
    Classes_DF = pd.DataFrame(columns=["ClassName","ChildClasses","ParintClasses","IndividualsClasses",
                                       "NamePropertis","SecondClass","TypeOuerClass"])
    NameClasses = []
    ChildClasses = []
    ParintClasses = []
    IndividualsClasses = []
    for cl in onto.classes():
        NameClasses.append(str(cl).split(".")[1])
        temp1 = []
        if len(onto.get_children_of(cl))!=0:

            for i in onto.get_children_of(cl):
                temp1.append(str(i).split(".")[1])
        else :
            temp1.append("none")
        temp2 = []
        if len(onto.get_instances_of(cl)) != 0:
            for i in onto.get_instances_of(cl):
                temp2.append(str(i).split(".")[1])  
        else:
            temp2.append("none")
        temp3 = []
        if len(cl.is_a) != 0:
            for i in cl.is_a:
                temp3.append(str(i).split(".")[1])  
        else:
            temp3.append("none")
        ChildClasses.append(temp1)
        IndividualsClasses.append(temp2)
        ParintClasses.append(temp3)
        
    
    Classes_DF['ClassName'] = NameClasses
    Classes_DF['ChildClasses'] = ChildClasses
    Classes_DF['ParintClasses'] = ParintClasses
    Classes_DF['IndividualsClasses'] = IndividualsClasses
    Classes_DF['NamePropertis'].fillna(str(["none"]),inplace = True)
    Classes_DF['SecondClass'].fillna(str(["none"]),inplace = True)
    Classes_DF['TypeOuerClass'].fillna(str(['none']),inplace = True)
    
    return Classes_DF

In [6]:
NameClasses = GetDataFrame(o1)

In [8]:
NameClasses.head(6)

,ClassName,ChildClasses,ParintClasses,IndividualsClasses,NamePropertis,SecondClass,TypeOuerClass
0,categories,"[supervised, unsupervised]",[ML],[none],['none'],['none'],['none']
1,supervised,"[classification, regression]",[categories],[none],['none'],['none'],['none']
2,unsupervised,"[clusturing, dimensionlity_reduction]",[categories],[none],['none'],['none'],['none']
3,classification,[none],[supervised],"[decision_tree, voting]",['none'],['none'],['none']
4,clusturing,[none],[unsupervised],"[cmeans, kmeans]",['none'],['none'],['none']
5,dimensionlity_reduction,[none],[unsupervised],[autoencoder],['none'],['none'],['none']


In [9]:
def iSindivisiual(onto,j):
    indivisual = list(onto.individuals())
    if len(indivisual)>0:
        for i in indivisual:
            if str(i).split(".")[1] == str(j).split(".")[1]:
                tepm = []
                for k in i.is_instance_of:
                    tepm.append(str(k).split(".")[1])
                return True , tepm
    return False ,[]

In [29]:
def iSclass(onto,j):
    classes = list(onto.classes())
    if len(str(j).split("."))>1:
        for i in classes:
            if str(i).split(".")[1] == str(j).split(".")[1]:
                return True
    return False

In [30]:
def iSclassFromName(onto,j):
    classes = list(onto.classes())
    for i in classes:
        if str(i).split(".")[1] == j:
            return True
    return False

In [31]:
def iSdatatype(j):
    if len(j)>6:
        return ["string"]
    return [j]

In [163]:
def processing_1(onto,j):
    isind,nameclass = iSindivisiual(onto,j)
    if not isind:
        nameclass = [str(j).split(".")[1]]
    return nameclass

In [164]:
def processing_2(onto,j):
    if len(str(j).split("."))>1:
        isind,nameclass = iSindivisiual(onto,j)
        if not isind:
            if iSclass(onto,j):
                nameclass = str(j).split(".")[1]
            else:
                nameclass = iSdatatype(j)
    else:
        nameclass = iSdatatype(j)
    return nameclass

In [165]:
def CombletDataFranClasses(onto,x,y,namepro,lable , list_prop , list_secondClass , list_label): # x is list of classes
    if iSclassFromName(onto,x):
        list_prop.append(namepro)  # must append
        list_secondClass.append(y)  # must append  #y is list
        list_label.append(lable)   # must append
            
    return list_prop , list_secondClass , list_label

In [171]:
def getRelations(onto):
    ClassFram = GetDataFrame(onto)
    
    for i in range( len(list(onto.classes()))) :
        className = list(onto.classes())[i].name
        list_prop =[] 
        list_secondClass =[] 
        list_label =[]
        for pro in onto.properties():
            relation = list(pro.get_relations())
            if len(relation)>0:
                for j in relation:
                    subject = []
                    objecte = []
                    subject= (processing_1(onto,j[0]))
                    objecte= (processing_2(onto,j[1]))
                    if className in subject:
                        list_prop , list_secondClass , list_label = CombletDataFranClasses(onto,className,objecte,str(pro).split(".")[1],1  , list_prop , list_secondClass , list_label)
                    if className in objecte:
                        list_prop , list_secondClass , list_label = CombletDataFranClasses(onto,className,subject,str(pro).split(".")[1],2  , list_prop , list_secondClass , list_label)

        if len(list(list_prop)) > 0:
            l1=[]
            l1.append(list_prop)
            ClassFram['NamePropertis'][ClassFram.loc[ClassFram['ClassName'] == className].index] = l1 # must append
        if len(list(list_secondClass)) > 0:
            l2=[]
            l2.append(list_secondClass)
            ClassFram['SecondClass'][ClassFram.loc[ClassFram['ClassName'] == className].index] = l2 # must append  #y is list
        if len(list(list_label)) > 0:
            l3=[]
            l3.append(list_label)
            ClassFram['TypeOuerClass'][ClassFram.loc[ClassFram['ClassName'] == className].index] = l3  # must append

    return ClassFram

In [172]:

def get_word2vec_sim_word_wod(w1, w2):
    try:
        sim = model.similarity(w1, w2)
    except: 
        sim = 0
    
    return sim

def  get_word2vec_sim_wo_List(w1 , lis):
    sim = 0
    for w in lis:
        s = get_word2vec_sim_word_wod(w1, w)
        if s > sim :
            sim = s
    
    return sim
    
    

def get_sim_name(name1, name2):
    sim = get_word2vec_sim_word_wod(name1, name2 )
    
    return sim

def get_sim_parent(l_pa1, l_pa2):
    sim = 0
    if l_pa1[0] != 'none':
        if l_pa2[0] != 'none':
            for w1 in l_pa1:
                maxSim = 0
                for w2 in l_pa2:
                    sim2 = get_word2vec_sim_word_wod(w1, w2)
                    if sim2 > maxSim:
                        maxSim = sim2
                sim = sim + maxSim
            sim = sim / len(l_pa1)
    
    return sim

def get_sim_child(l_ch1, l_ch2):
    sim = 0
    if l_ch1[0] != 'none':
        if l_ch2[0] != 'none':
            for w1 in l_ch1:
                maxSim = 0
                for w2 in l_ch2:
                    sim2 = get_word2vec_sim_word_wod(w1, w2)
                    if sim2 > maxSim:
                        maxSim = sim2
                sim = sim + maxSim
            sim = sim / len(l_ch1)
    
    return sim

def get_sim_indv(l_indv1, l_indv2):
    sim = 0
    if l_indv1[0] != 'none':
        if l_indv2[0] != 'none':
            for w1 in l_indv1:
                maxSim = 0
                for w2 in l_indv2:
                    sim2 = get_word2vec_sim_word_wod(w1, w2)
                    if sim2 > maxSim:
                        maxSim = sim2
                sim = sim + maxSim
            sim = sim / len(l_indv1)
    
    return sim



def  get_ind_word2vec_sim_propa_ListPropa(propa , l_propar2 ):
    ind = 0
    sim = 0 
    for w in l_propar2:
        s = get_word2vec_sim_word_wod(propa, w)
        if s > sim :
            ind = (l_propar2.index(w))
            sim = s
    
    return ind , sim

def get_sim_propar(l_propar1 , l_l_Second_Party1 , label1  ,  l_propar2 , l_l_Second_Party2 , label2):
    simil = 0
    if l_propar1[0] != 'none':
        if l_propar2[0] != 'none':
            for propa in l_propar1:
                ind , sim = get_ind_word2vec_sim_propa_ListPropa(propa , l_propar2 )
                simil = simil + sim
                if(label1[l_propar1.index(propa)] == label2[ind]):
                   sim2 = get_sim_parent(l_l_Second_Party1[l_propar1.index(propa)], l_l_Second_Party2[ind])
                   simil = simil + sim
                else: 
                   simil = simil - sim
            return simil / len(l_propar1)
        
    
    
def get_collec_sim(ClassName ,ChildClass ,ParintClass ,IndividualsClass ,NamePropert ,SecondCla ,TypeOuerCla   , ClassName2 ,ChildClass2 ,ParintClass2 ,IndividualsClass2 ,NamePropert2 ,SecondCla2 ,TypeOuerCla2):
    sim1 = get_sim_name(ClassName , ClassName2)
    sim2 = get_sim_child(ChildClass , ChildClass2)
    sim3 = get_sim_parent(ParintClass , ParintClass2) 
    sim4 = get_sim_indv(IndividualsClass , IndividualsClass2)
    sim5 = get_sim_propar(NamePropert ,SecondCla ,TypeOuerCla ,  NamePropert2 ,SecondCla2 ,TypeOuerCla2)    
    
    sim = sim1+sim2+sim3+sim4+sim5
    sim = sim/5
    return sim *10

In [175]:
dataFrame = getRelations(o1)
dataFrame2 = getRelations(AllOnto[2])

In [176]:
dataFrame

,ClassName,ChildClasses,ParintClasses,IndividualsClasses,NamePropertis,SecondClass,TypeOuerClass
0,categories,"[supervised, unsupervised]",[ML],[none],['none'],['none'],['none']
1,supervised,"[classification, regression]",[categories],[none],['none'],['none'],['none']
2,unsupervised,"[clusturing, dimensionlity_reduction]",[categories],[none],['none'],['none'],['none']
3,classification,[none],[supervised],"[decision_tree, voting]",['none'],['none'],['none']
4,clusturing,[none],[unsupervised],"[cmeans, kmeans]",['none'],['none'],['none']
5,dimensionlity_reduction,[none],[unsupervised],[autoencoder],['none'],['none'],['none']
6,regression,[none],[supervised],"[Linear_Regression, Logistic_Regression]",['none'],['none'],['none']
7,ML,"[categories, applications, problem, step, tool...",[Thing],[none],['none'],['none'],['none']
8,applications,[none],[ML],"[Image_classification, Information_extraction,...",['none'],['none'],['none']
9,problem,[none],[ML],"[Bais, Underfitting, Variance, capacity, overf...","[discribtion, discribtion, discribtion, discri...","[[string], [string], [string], [string], [stri...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [170]:
dataFrame.shape

(18, 7)

In [157]:
dataFrame2.head(5)

,ClassName,ChildClasses,ParintClasses,IndividualsClasses,NamePropertis,SecondClass,TypeOuerClass
0,Input,[Dataset],[Task],"[Input, Input0, Input1, Input1, Input2, Input3]","[sample, sample, sample, sample, sample, sampl...","[[string], [string], [string], [string], [stri...","[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, ..."
1,Dataset,[none],[Input],"[Mall_Customers, Real_estate, weather-stations...","[description, description, synonym, synonym, c...","[[string], [string], [string], [string], [Inpu...","[1, 1, 1, 1, 1, 1, 1, 1]"
2,Algorithm,"[Operator, HyperParameter]",[Thing],"[DBSCAN, Optics]","[isSupervised, isSupervised, name, name, synon...","[[No], [No], [string], [string], [string], [st...","[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,Operator,[none],[Algorithm],"[The_first_step, The_second_step, The_third_step]","[description, description, description, synony...","[[string], [string], [string], [string], [stri...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2]"
4,Task,"[Input, Output]",[Thing],"[House_price, Mall_Customers, Werther_Stations...","[description, description, description, synony...","[[string], [string], [string], [string], [stri...","[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, ..."


In [158]:
def get_sim_dataFrame (dataFrame , dataFrame2):
    li_c1 = []
    li_c2 = []
    li_sim = []
    ClassNamelist = dataFrame["ClassName"].values
    ChildClasses = dataFrame["ChildClasses"].values
    ParintClasses = dataFrame["ParintClasses"].values
    IndividualsClasses = dataFrame["IndividualsClasses"].values
    NamePropertis = dataFrame["NamePropertis"].values
    SecondClass = dataFrame["SecondClass"].values
    TypeOuerClass = dataFrame["TypeOuerClass"].values

    ClassNamelist2 = dataFrame2["ClassName"].values
    ChildClasses2 = dataFrame2["ChildClasses"].values
    ParintClasses2 = dataFrame2["ParintClasses"].values
    IndividualsClasses2 = dataFrame2["IndividualsClasses"].values
    NamePropertis2 = dataFrame2["NamePropertis"].values
    SecondClass2 = dataFrame2["SecondClass"].values
    TypeOuerClass2 = dataFrame2["TypeOuerClass"].values
    for i in range(dataFrame.shape[0]):
        ClassName = ClassNamelist[i]
        ChildClass = ChildClasses[i]
        ParintClass = ParintClasses[i]
        IndividualsClass = IndividualsClasses[i]
        NamePropert = NamePropertis[i]
        SecondCla = SecondClass[i]
        TypeOuerCla = TypeOuerClass[i]
        sim =0 
        ClassName22 = ''
        for i2 in range(dataFrame2.shape[0]):
            ClassName2 = ClassNamelist2[i2]
            ChildClass2 = ChildClasses2[i2]
            ParintClass2 = ParintClasses2[i2]
            IndividualsClass2 = IndividualsClasses2[i2]
            NamePropert2 = NamePropertis2[i2]
            SecondCla2 = SecondClass2[i2]
            TypeOuerCla2 = TypeOuerClass2[i2]

            sim2 = get_collec_sim(ClassName ,ChildClass ,ParintClass ,IndividualsClass ,NamePropert ,SecondCla ,TypeOuerCla   , ClassName2 ,ChildClass2 ,ParintClass2 ,IndividualsClass2 ,NamePropert2 ,SecondCla2 ,TypeOuerCla2)
            if sim2 > sim:
                sim = sim2
                ClassName22 = ClassName2
        li_c1.append(ClassName)
        li_c2.append(ClassName22)
        li_sim.append(sim)

    return li_c1 , li_c2 , li_sim




In [177]:
li_c1 , li_c2 , li_sim = get_sim_dataFrame (dataFrame , dataFrame2)

In [178]:
print(li_c1)
print('=============================================================')
print(li_c2)
print('=============================================================')
print(li_sim)

['categories', 'supervised', 'unsupervised', 'classification', 'clusturing', 'dimensionlity_reduction', 'regression', 'ML', 'applications', 'problem', 'step', 'Feature_Extraction', 'Feature_Selection', 'Model_evaluation', 'Model_selection', 'data_collection', 'fit_model', 'tools_and_librarys']
['Algorithm', 'Task', 'Operator', 'Input', 'Dataset', 'Dataset', 'Algorithm', 'Task', 'Dataset', 'Task', 'Input', 'Dataset', 'Algorithm', 'Output', 'Algorithm', 'Algorithm', 'Algorithm', 'Operator']
[0.5007066521793604, 0.5087976455688477, 0.3443267494440079, 0.5321995541453362, 0.03703208267688751, 0.03703208267688751, 1.0016347169876099, 2.387147864699364, 0.7339242150386174, 0.5173184424638748, 0.5450093746185303, 1.2668635249137878, 0.25805744528770447, 0.4842716405789057, 0.25805744528770447, 0.9535462558269501, 0.25805744528770447, 0.17858898639678955]


In [179]:
sim_classes = []
for i in range(len(list(li_c1))):
    if li_sim[i] > 0.7:
        sim_classes.append([li_c1[i],li_c2[i],li_sim[i]])

In [180]:
for l in sim_classes:
    print(l)
    print('==================')

['regression', 'Algorithm', 1.0016347169876099]
['ML', 'Task', 2.387147864699364]
['applications', 'Dataset', 0.7339242150386174]
['Feature_Extraction', 'Dataset', 1.2668635249137878]
['data_collection', 'Algorithm', 0.9535462558269501]
